#ESTIM



---





This Python Notebook will walk you through applying different machine learning models to your own dataset (should update after writing abstract)


##Inputting Your Data

---



Put all of your data in the data folder

In [ ]:
data_dir = "/content/drive/My Drive/estim/data"

##Importing Packages

---





> General Imports:







In [ ]:
import math, csv, random, time
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
import glob
from collections import defaultdict
import matplotlib.pyplot as plt



> Model Imports:

 NOTE: LIKELY MISSING SOME


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import chi2

> Evaluation Imports:

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV

>Mounting Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Intra-subject

>MLP:

In [ ]:
### Load data from csv file
def loadCsv(filename):
    lines = csv.reader(open(filename, newline= '', encoding='utf-8-sig'), delimiter=',', quotechar='|')
    dataset = []
    for row in lines:
        if row[12] != "": # checking if the last cell in each row (column 12) is not empty
            dataset.append([float(x) for x in row])
    return dataset

### Construct data frame
def data(filename, scale):
    
    #Load data from file
    dataset = np.array(loadCsv(filename))
    indices_control_or_other = []

    # find rows that are Control (0) or Other (9)
    for i in range(len(dataset)):
        if dataset[i][1] == 0 or dataset[i][1] == 9:
            indices_control_or_other.append(i)
    
    dataset = np.delete(dataset, indices_control_or_other, 0) # delete rows that are Control/Other
    dataset = np.delete(dataset,[0, 11], 1) # deleting columns 0 and 11

    row_len = len(dataset[0])
    X = np.array(dataset[:, :row_len-1])
    y = np.array(dataset[:, row_len-1])
    # Cap ground truth within [0, 1]
    for i, truth in enumerate(y):
      if truth > 1:
        y[i] = 1
      if truth < 0:
        y[i] = 0

    #Standardize and scale data
    if (scale):
        X = preprocessing.scale(X)
    return X, y

### Evaluate model
def evaluate(model_id, X, y, scale, seed=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    index_cap = len(y_train)

    all_index = []
    index_10 = random.sample(range(index_cap),10)
    all_index.append(index_10)
    index_20 = random.sample(range(index_cap),20)
    all_index.append(index_20)
    index_50 = random.sample(range(index_cap),50)
    all_index.append(index_50)
    index_100 = random.sample(range(index_cap),100)
    all_index.append(index_100)
    index_200 = random.sample(range(index_cap),200)
    all_index.append(index_200)

    for i in range(1):
        if (index_cap) > 500:
            index_500 = random.sample(range(index_cap),500)
            all_index.append(index_500)
        else:
            index_500 = random.sample(range(index_cap),index_cap)
            all_index.append(index_500)
            break
        if (index_cap) > 1000:
            index_1000 = random.sample(range(index_cap),1000)
            all_index.append(index_1000)
        else:
            index_1000 = random.sample(range(index_cap),index_cap)
            all_index.append(index_1000)
            break
        if (index_cap) > 2000:
            index_2000 = random.sample(range(index_cap),2000)
            all_index.append(index_2000)
        else:
            index_2000 = random.sample(range(index_cap),index_cap)
            all_index.append(index_2000)
            break

  for i, indices in enumerate(all_index):
    length = len(indices)
    print("Fitting model parameters on training set")
    t0 = time.time()
    grid = {'activation': ['identity','logistic', 'tanh', 'relu'], 'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'hidden_layer_sizes': [(10, ), (20, ), (30, ), (40, ), (50, ), (10, 10, ), (20, 10, )], }
    clf = GridSearchCV(MLPRegressor(max_iter = 1000, solver = 'lbfgs' , random_state=seed), param_grid=grid, cv=5, iid=False, scoring='neg_mean_squared_error')
    clf.fit(X_train, y_train)
    print("done in %0.3fs" % (time.time() - t0))
    print("\nBest estimator found by grid search:")
    print('\t'+str(clf.best_estimator_))

    print("\nEvaluating best estimator on test set")
    t0 = time.time()
    y_pred = clf.predict(X_test)
    # cap predictions within [0, 1]
    for i, pred in enumerate(y_pred):
      if pred > 1:
        y_pred[i] = 1
      if pred < 0:
        y_pred[i] = 0
    print("done in %0.3fs" % (time.time() - t0))

    score = round(mean_absolute_error(y_test, y_pred), 4)
    print('\n\t\tMAE (test):', score)
    scores_MLP_intra[i].append(score)


scores_10 = []
scores_20 = []
scores_50 = []
scores_100 = []
scores_200 = []
scores_500 = []
scores_1000 = []
scores_2000 = []

scores_MLP_intra = [scores_10, scores_20, scores_50, scores_100, scores_200, scores_500, scores_1000, scores_2000]
experiment_strings = ["10", "20", "50", "100", "200", "500", "1000", "2000"]

test_files = []


#### Methods to run:
for filepath in glob.iglob(data_dir + '/*.csv'):
    X, y = data(filepath, True)
    print("Evaluating:" + filepath)
    evaluate(id, X, y, True, 42)

In [ ]:
### Plotting code for MLP intrasubject incremental
# mean across test files
mean_scores_MLP_intra = [np.mean(experiment_scores) for experiment_scores in scores_MLP_intra]

plt.plot(experiment_strings, mean_scores_MLP_intra)
plt.table(experiment_strings, mean_scores_MLP_intra)

> Random Forest:

##Cross-Subject

>MLP:

> Random Forest:

##Incremental Pre-Trained

>MLP:

> Random Forest:

##Active Learning

>MLP:

> Random Forest:

In [ ]:
# Active Learning Imports
from modAL.models import ActiveLearner

#### USER PARAMETERS: ####
# Number of trials to run
num_trials = 20

# Number of rows for inital active learner training
num_init_rows = 5

# Directory with CSV files
data_dir = "data"

# Test ratio (for train-test split)
test_ratio = 0.2
#########################

### Load data from csv file
def loadCsv(filename):
    lines = csv.reader(open(filename, newline= '', encoding='utf-8-sig'), delimiter=',', quotechar='|')
    dataset = []
    for row in lines:
        if row[12] != "": # checking if the last cell in each row (column 12) is not empty
            dataset.append([float(x) for x in row])
    return dataset

### Construct data frame
def data(filename, scale):

    dataset = np.array(loadCsv(filename))
    
    indices_control_or_other = []

    # find rows that are Control (0) or Other (9)
    for i in range(len(dataset)):
        if dataset[i][1] == 0 or dataset[i][1] == 9:
            indices_control_or_other.append(i)
    
    dataset = np.delete(dataset, indices_control_or_other, 0) # delete rows that are Control/Other
    dataset = np.delete(dataset,[0, 11], 1) # deleting columns 0 and 11

    row_len = len(dataset[0])
    X = np.array(dataset[:, :row_len-1])
    y = np.array(dataset[:, row_len-1])

    # Cap ground truth within [0, 1]
    for i, truth in enumerate(y):
        if truth > 1:
            y[i] = 1
        if truth < 0:
            y[i] = 0
            
    #Standardize and scale data
    if (scale):
        X = preprocessing.scale(X)
    return X, y

### RF Query Strategy (max standard deviation over tree predictions)
def RF_std_query(active_learer, X):
    RF = active_learer.estimator
    per_tree_pred = np.array([tree.predict(X) for tree in RF.estimators_])
    std_arr = np.std(per_tree_pred, axis=0) # std of predictions over all trees 
    query_idx = np.argmax(std_arr)
    return query_idx, X[query_idx]

### Evaluate model
def evaluate(model_id, X, y, scale=False, seed=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=seed)

    # choose a small amount of random rows for initial training (e.g. 5)
    init_indices = np.random.choice(index_cap, num_init_rows, replace=False)
    print(f"Indices of intial train data: {init_indices}")
    init_X = X_train[init_indices]
    init_y = y_train[init_indices]

    # remove the initial rows from our train data
    X_train = np.delete(X_train, init_indices, axis=0)
    y_train = np.delete(y_train, init_indices, axis=0)

    for num_train in experiments:
        num_queries = num_train - num_init_rows

        #t0 = time.time()
        learner = ActiveLearner(estimator=RandomForestRegressor(max_features="auto", criterion="mse",
         n_estimators=100, random_state=42), query_strategy=RF_std_query, X_training=init_X, y_training=init_y)

        for _ in range(num_queries):
            query_idx, query_instance = learner.query(X_train)
            print(query_idx)
            learner.teach(X_train[query_idx].reshape(1, -1), np.array([y_train[query_idx]]))

        y_pred = learner.predict(X_test)
        #print("done in %0.3fs" % (time.time() - t0))

        # cap predictions within [0, 1]
        for i, pred in enumerate(y_pred):
          if pred > 1:
            y_pred[i] = 1
          if pred < 0:
            y_pred[i] = 0

        score = round(mean_absolute_error(y_test, y_pred), 4)
        
        # add score for this trial to results dictionary in correct experiment row
        results[num_train].append(score)

        print(f"MAE (test) with {num_train} rows: {score}")

def to_num(label):
    # remove all non-numeric characters
    label_str = ""
    for char in label:
        if char.isdigit():
            label_str += char
    return label_str

### Main Loop (Over Files)###
print(f"Active Learning with Random Forest Regressor.")
print(f"Model trained with {num_init_rows} rows before querying starts.")

for filepath in glob.iglob(data_dir + '/*.csv'):
    X, y = data(filepath, True)
    
    index_cap = int(len(X) * (1 - test_ratio))

    # num of training rows
    experiments = [10, 20, 50, 100, 200] 
    
    # add experiments for {500, 1000} if there are enough rows 
    if index_cap > 500:
        experiments.append(500)
    if index_cap > 1000:
        experiments.append(1000)

    # always run an experiment with all data
    experiments.append(index_cap)

    # results dictionary, with each key referring to a list of scores for that experiment
    results = defaultdict(list)

    # RUN TRIALS
    for trial in range(1, num_trials + 1):
        print(f"FILE {filepath}. TRIAL {trial}/{num_trials}.")
        evaluate(id, X, y, True, 42)

    # convert to DataFrame for csv saving
    results_df = pd.DataFrame.from_dict(results, orient="index")

    results_df.to_csv(f"RF_ActiveLearn_{to_num(filepath)}.csv")